In [0]:
#!import "../DataModel/DataStructure"
using System.Linq.Expressions;
using static Systemorph.Vertex.Equality.IdentityPropertyExtensions;

In [0]:
public class IdentityPropertyReader<T> where T:BaseVariableIdentity {
    private static Func<T, string> identityReader;
    
    private IdentityPropertyReader(){
        identityReader = GetIdentityReader();
    }
    public static readonly IdentityPropertyReader<T> Instance = new IdentityPropertyReader<T>();

    private Func<T, string> GetIdentityReader(){
        var parameter = Expression.Parameter(typeof(T));
        var identityProperties = typeof(T).GetIdentityProperties().Select(x=>x.Name).Where(x=> x != "AocType" && x != "Novelty").ToArray();
        var expressionstToConcat = identityProperties.SelectMany(x=> new Expression[]{Expression.Constant(x+": "), Expression.Property(parameter, x) })
                                                     .ToArray();
        var partialConcatExpression = Expression.Call(typeof(IdentityPropertyReader<T>).GetMethod("concat"), expressionstToConcat[0], expressionstToConcat[1]);
        for (int i = 2; i < expressionstToConcat.Count(); i++){
            if(i%2==0) partialConcatExpression = Expression.Call(typeof(IdentityPropertyReader<T>).GetMethod("concat"), partialConcatExpression, Expression.Constant(", "));
            if(i%2==1 && identityProperties[(int)Math.Floor((double)i/2)] == "AccidentYear"){
                partialConcatExpression = Expression.Call(typeof(IdentityPropertyReader<T>).GetMethod("concat2"), partialConcatExpression, expressionstToConcat[i]);   
            } else{
                partialConcatExpression = Expression.Call(typeof(IdentityPropertyReader<T>).GetMethod("concat"), partialConcatExpression, expressionstToConcat[i]);   
            }
        } // i needed (imo) to do it like this, since int? gives problem as Object...
        return Expression.Lambda<Func<T,string>>(partialConcatExpression, parameter).Compile();
    }
    public string GetIdentityString(T x) => identityReader(x);
    
    public static string concat(Object first, Object second) => first.ToString()+second.ToString();
    
    public static string concat2(Object first, int? second){
        if(second == null) return first.ToString();
        return first.ToString()+second.ToString();
    }
}